In [ ]:
conda create -n face_net python=3.8
conda activate face_net
conda create ipykernal
conda install tensorflow-gpu==2.4.1

## 테스트 데이터셋 추출
#### Train dataset 폴더에서 각 한장씩을 뽑아서 테스트 데이터셋으로 생성

In [62]:
import pathlib
import os
import shutil

TRAIN_FOLDER_NAME='data/images/train_celeb' # 학습 데이터 위치
TEST_FOLDER_NAME='data/images/test_celeb'


train_data_dir = pathlib.Path('./'+TRAIN_FOLDER_NAME)
test_data_dir = pathlib.Path('./'+TEST_FOLDER_NAME)

print("학습 이미지 경로 >> "+str(train_data_dir))
print("테스트 이미지 저장 경로 >> "+str(train_data_dir))

class_name_list = []


# 테스트 데이터 폴더 생성
if not os.path.exists(test_data_dir): 
    os.makedirs(test_data_dir)

# 학습 데이터셋에서 한장씩 뽑아서 train dataset으로 넣음
for i,entry in enumerate(os.scandir(train_data_dir)):
    class_name_list.append(entry.name)
    
    target_path = str(test_data_dir)+"/"+entry.name
    if not os.path.exists(target_path): 
        print("폴더 생성 >> "+target_path)
        os.makedirs(target_path)  

    image_list = os.listdir(entry.path)  
    shutil.move(entry.path+"/"+image_list[0], target_path+"/"+image_list[0])




학습 이미지 경로 >> data/images/train_celeb
테스트 이미지 저장 경로 >> data/images/train_celeb


## 학습 데이터셋 얼굴영역 추출
##### export PYTHONPATH=/data/itrnd/Face_Recog/facenet/src

In [68]:
%env PYTHONPATH=/data/itrnd/Face_Recog/facenet/src

env: PYTHONPATH=/data/itrnd/Face_Recog/facenet/src


In [69]:
!python ./src/align/align_dataset_mtcnn.py \
./data/images/train_celeb \
./data/images/train_celeb_aligned \
--image_size 160

2022-11-03 14:08:01.383648: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Creating networks and loading parameters
2022-11-03 14:08:02.369091: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-03 14:08:02.370429: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-03 14:08:02.371206: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-11-03 14:08:02.404760: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA 

## 테스트 데이터셋 얼굴영역 추출

In [71]:
!python ./src/align/align_dataset_mtcnn.py \
./data/images/test_celeb \
./data/images/test_celeb_aligned \
--image_size 160

2022-11-03 14:17:15.592161: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Creating networks and loading parameters
2022-11-03 14:17:16.499321: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-03 14:17:16.500536: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-03 14:17:16.501671: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-11-03 14:17:16.537657: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA 

## 얼굴 임베딩 학습

In [72]:
!python ./src/classifier.py TRAIN \
./data/images/train_celeb_aligned/ \
./models/20180402-114759.pb \
./models/celeb_classifier.pkl


./src/classifier.py:59: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(cls.image_paths)>0, 'There must be at least one image for each class in the dataset')
2022-11-03 14:19:01.522019: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-11-03 14:19:02.230863: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-03 14:19:02.231312: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-11-03 14:19:02.257671: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 14:19:02.258396: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX

## 데이터 분류

In [74]:
!python ./src/classifier.py CLASSIFY \
./data/images/test_celeb_aligned/ \
./models/20180402-114759.pb \
./models/celeb_classifier.pkl

./src/classifier.py:59: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(cls.image_paths)>0, 'There must be at least one image for each class in the dataset')
2022-11-03 14:30:42.898141: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-11-03 14:30:43.606980: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-03 14:30:43.607430: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-11-03 14:30:43.633743: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 14:30:43.634492: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX